# Import

In [1]:
import pandas as pd
from sodapy import Socrata
from api_keys import *
import datetime

In [2]:
# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("www.dallasopendata.com", app_token)

In [3]:
#grab data from api
#sdr7-6v3j is the dataset code from sodapy
all_data = client.get_all("sdr7-6v3j")

#turn into pandas df
data_df = pd.DataFrame.from_records(all_data)

In [4]:
data_df.sort_values(by='ararrestdate',ascending=False, inplace=True)
# data_df

In [5]:
# data_df.columns
#looking for only certain columns
#'incidentnum', 'arrestnumber', 'ararrestdate', 'ararresttime', 'arpremises', 'arladdress', 'arlzip','sex','drugrelated','drugtype','age'
data_df = data_df[['incidentnum', 'arrestnumber', 'ararrestdate', 'ararresttime', 'arpremises', 'arladdress', 'arlzip','sex','drugrelated','drugtype','age']]
# data_df.head()

In [6]:
data_df = data_df.loc[((data_df['drugrelated']=='Yes') | (data_df['drugrelated']=='Uknown'))]
# data_df.head()

In [7]:
for index, row in data_df.iterrows():
        split = row['ararrestdate'].split('T')
        row['ararrestdate'] = split[0]


In [8]:
data_df.head()

,incidentnum,arrestnumber,ararrestdate,ararresttime,arpremises,arladdress,arlzip,sex,drugrelated,drugtype,age
881,201734-2022,22-031919,2022-11-07,04:06,Apartment Residence,3637 TRINITY MILLS RD,75287,Male,Uknown,NaN,30
834,201702-2022,22-031908,2022-11-07,02:35,"Highway, Street, Alley ETC",3510 COMMERCE ST,75226,Male,Uknown,NaN,44
826,202171-2022,22-031965,2022-11-07,18:59,"Highway, Street, Alley ETC",1300 TRAYMORE AVE,75217,Female,Yes,Other Non-Prescription Drugs,56
844,201429-2022,22-031891,2022-11-06,15:17,Apartment Residence,14501 MONTFORT DR,75254,Male,Yes,Methamphetamine,52
827,201068-2022,22-031798,2022-11-06,01:23,NaN,400 CRESCENT CT,75201,Male,Uknown,NaN,54


In [9]:
import sqlite3

In [10]:
conn =sqlite3.connect('test_database')
c = conn.cursor()

In [11]:
c.execute('CREATE TABLE IF NOT EXISTS crime_test (incidentnum, arrestnumber, ararrestdate, ararresttime, arpremises, arladdress, arlzip, sex, drugrelated, drugtype, age)')
conn.commit()

In [12]:
data_df.to_sql('crime_test', conn, if_exists='replace', index = False)

20618

In [13]:
c.execute('''select * from crime_test''')

In [14]:
for row in c.fetchall():
    print(row)
    break

('201734-2022', '22-031919', '2022-11-07', '04:06', 'Apartment Residence', '3637 TRINITY MILLS RD', '75287', 'Male', 'Uknown', None, '30')


In [15]:
#decrease the amount of data to filter by today or past week, month, and year
#creating the variables to filter through the df's
today = datetime.date.today()
week = today - datetime.timedelta(days=7)
month = today - datetime.timedelta(days=30)
year = (today - datetime.timedelta(days=365)).strftime('%Y-%m-%d')
today = today.strftime('%Y-%m-%d')
week = week.strftime('%Y-%m-%d')
month = month.strftime('%Y-%m-%d')

In [16]:
#filter through the dataframes descending to not constantly work with big data for each filter
year_df = data_df[(data_df['ararrestdate'] >= year)]
month_df = year_df[(year_df['ararrestdate'] >= month)]
week_df = month_df[(month_df['ararrestdate'] >= week)]
today_df = week_df[(week_df['ararrestdate'] >= today)]

In [17]:
week_df

,incidentnum,arrestnumber,ararrestdate,ararresttime,arpremises,arladdress,arlzip,sex,drugrelated,drugtype,age
881,201734-2022,22-031919,2022-11-07,04:06,Apartment Residence,3637 TRINITY MILLS RD,75287,Male,Uknown,NaN,30
834,201702-2022,22-031908,2022-11-07,02:35,"Highway, Street, Alley ETC",3510 COMMERCE ST,75226,Male,Uknown,NaN,44
826,202171-2022,22-031965,2022-11-07,18:59,"Highway, Street, Alley ETC",1300 TRAYMORE AVE,75217,Female,Yes,Other Non-Prescription Drugs,56
844,201429-2022,22-031891,2022-11-06,15:17,Apartment Residence,14501 MONTFORT DR,75254,Male,Yes,Methamphetamine,52
827,201068-2022,22-031798,2022-11-06,01:23,NaN,400 CRESCENT CT,75201,Male,Uknown,NaN,54
768,201056-2022,22-031787,2022-11-06,00:05,Outdoor Area Public/Private,2700 ELM ST,75226,Female,Uknown,NaN,34
842,201429-2022,22-031877,2022-11-06,15:17,Apartment Residence,14501 MONTFORT DR,75254,Female,Yes,Other Prescription Drugs,51
793,201131-2022,22-031829,2022-11-06,03:20,"Highway, Street, Alley ETC",5400 BRUTON RD,75217,Male,Uknown,NaN,NaN
865,201429-2022,22-031890,2022-11-06,15:17,Apartment Residence,14501 MONTFORT DR,75254,Male,Yes,Methamphetamine,37
792,201142-2022,22-031816,2022-11-06,03:00,"Highway, Street, Alley ETC",3200 TUMALO TRL,75212,Male,Yes,Cultivated Marijuana,29


In [18]:
from geopy.geocoders import Nominatim


In [43]:
geolocator = Nominatim(user_agent="test_crime_data_app")
location = geolocator.geocode("8550 N STEMMONS SERV NB")

print((location.latitude, location.longitude))

AttributeError: 'NoneType' object has no attribute 'latitude'

In [62]:
# loop through each rows address and zip code, IF address + zip code gives lat/long append, ELIF address gives lat/long append, elif get zip code lat/lng, else no zip/address put NaN for lat/lng
lat_lngs = {'lat':[],'lng':[]}

In [63]:
for index, row in year_df.iterrows():
    test = ''
    test += row['arladdress']
    test += ' ' + row['arlzip']
    try:
        location = geolocator.geocode(test)
        lat_lngs['lat'].append(location.latitude)
        lat_lngs['lng'].append(location.longitude)
    except:
        try:
            test = test[:-5]
            location = geolocator.geocode(test)
            lat_lngs['lat'].append(location.latitude)
            lat_lngs['lng'].append(location.longitude)
        except:
            try:
                location = geolocator.geocode(row['arlzip'])
                lat_lngs['lat'].append(location.latitude)
                lat_lngs['lng'].append(location.longitude)
            except:
                location = ['NaN', 'Nan']
                lat_lngs['lat'].append(location.latitude)
                lat_lngs['lng'].append(location.longitude)

In [64]:
lat_lng_df = pd.DataFrame(lat_lngs)

In [65]:
year_df.reset_index(inplace=True)

In [58]:
week_df.reset_index(inplace=True)

In [59]:
week_df.head()

,index,incidentnum,arrestnumber,ararrestdate,ararresttime,arpremises,arladdress,arlzip,sex,drugrelated,drugtype,age
0,881,201734-2022,22-031919,2022-11-07,04:06,Apartment Residence,3637 TRINITY MILLS RD,75287,Male,Uknown,NaN,30
1,834,201702-2022,22-031908,2022-11-07,02:35,"Highway, Street, Alley ETC",3510 COMMERCE ST,75226,Male,Uknown,NaN,44
2,826,202171-2022,22-031965,2022-11-07,18:59,"Highway, Street, Alley ETC",1300 TRAYMORE AVE,75217,Female,Yes,Other Non-Prescription Drugs,56
3,844,201429-2022,22-031891,2022-11-06,15:17,Apartment Residence,14501 MONTFORT DR,75254,Male,Yes,Methamphetamine,52
4,827,201068-2022,22-031798,2022-11-06,01:23,NaN,400 CRESCENT CT,75201,Male,Uknown,NaN,54


In [66]:
merged = pd.concat([year_df, lat_lng_df],axis=1)

In [71]:
test_df = merged.loc[merged['lat'] == 'NaN']

In [74]:
merged['lat'].unique()

array([32.9877421 , 32.78523115, 32.734616  , ..., 32.74900755,
       32.78197287, 32.8698453 ])